In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import sys
import argparse
import h5py
from scipy.stats import t as student_t
from statsmodels.stats import multitest as mt
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
vars_df = pd.read_pickle('../../agemapper/Analsysis Female New/female_vars_testB.pkl')
    
with open('../../agemapper/Analsysis Female New/varsHeader.txt') as f:
    varsHeader = f.readlines()
    varsHeader = [l.strip('\n\r') for l in varsHeader]
    varsHeader = np.array(varsHeader)
with open('../../agemapper/Analsysis Female New/vars_categories.txt') as f:
    vars_categories = f.readlines()
    vars_categories = [l.strip('\n\r') for l in vars_categories]
    vars_categories = np.array(vars_categories)
    
vars_df = vars_df[(vars_df.modality == 'T1_nonlinear') |
                (vars_df.modality == 'T2_nonlinear') |
                (vars_df.modality == 'tbss_FA') |
                  (vars_df.modality == 'tbss_MD') |
                (vars_df.modality == 'rsfmri_0') |
                (vars_df.modality == 'swi') |
                  (vars_df.modality == 'tfmri_1') |
                (vars_df.modality == 'tracts')   |
                (vars_df.modality == 'tbss_MO') |
                  (vars_df.modality == 'tbss_ISOVF_s') |
                (vars_df.modality == 'tbss_L2') |
                (vars_df.modality == 'rsfmri_2') |
                  (vars_df.modality == 'rsfmri_10') |
                (vars_df.modality == 'rsfmri_5') |
                (vars_df.modality == 'rsfmri_21') |
                  (vars_df.modality == 'T2_lesions') |
                (vars_df.modality == 'tbss_FA_s') |
                (vars_df.modality == 'tbss_L3_s') 
                  
]


vars_over_bThr_list = []

for idx in range(len(vars_df)):
    df = vars_df.iloc[idx].dataframe
    bThr = vars_df.iloc[idx].bonf
    vars_over_bThr = df[df.log_p_values>=bThr].sort_values('log_p_values', ascending=False).names.to_list()
    for var in vars_over_bThr:
        vars_over_bThr_list.append(var)
        
print("Number of unique vars across all modalities: ", len(set(vars_over_bThr_list)))

unique_vars_over_bTHr = list(set(vars_over_bThr_list))
print(len(unique_vars_over_bTHr))

unique_vars_over_bThr_categories = []

for var in unique_vars_over_bTHr:
    idx = list(varsHeader).index(var)
    category = vars_categories[idx]
    unique_vars_over_bThr_categories.append(category)
    
modalities = []
pearson_rs_list = []
logPs = []
for var in unique_vars_over_bTHr:
    modalities_with_var = []
    pearsonRs_with_var = []
    logPs_with_var = []
    for idx in range(len(vars_df)):
        df = vars_df.iloc[idx].dataframe
        modality = vars_df.iloc[idx].modality
        bThr = vars_df.iloc[idx].bonf
        df = df[df.log_p_values>=bThr].sort_values('log_p_values', ascending=False)
        vars_over_bThr = df.names.to_list()
        pearson_r_over_bThr = df.pearson_r.to_list()
        logps_over_bTHr = df.log_p_values.to_list()
        if var in vars_over_bThr:
            modalities_with_var.append(modality)
            pearsonRs_with_var.append(pearson_r_over_bThr[vars_over_bThr.index(var)])
            logPs_with_var.append(logps_over_bTHr[vars_over_bThr.index(var)])
    modalities.append(modalities_with_var)
    pearson_rs_list.append(pearsonRs_with_var)
    logPs.append(logPs_with_var)
    
df_vars_over_bThr = pd.DataFrame.from_dict({
    'names': unique_vars_over_bTHr,
    'Categories': unique_vars_over_bThr_categories,
    'modalities': modalities,
    'pearson_r': pearson_rs_list,
    'log_p_values': logPs
})

for cat in sorted(list(set(df_vars_over_bThr.Categories.to_list()))):
    print("Category: {}".format(cat))
    print('------------------------------------------------')
    names = df_vars_over_bThr[df_vars_over_bThr.Categories == cat].names.to_list()
    for name in sorted(names):
        print('Var: {}'.format(name))
        mods = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].modalities.to_list()
        prs = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].pearson_r.to_list()
        logps = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].log_p_values.to_list()
        print('   Modalities:')
        
        prs_abs = np.abs(prs).tolist()
        _, mods = zip(*sorted(zip(prs_abs[0], mods[0]), reverse=True))
        _, prs = zip(*sorted(zip(prs_abs[0], prs[0]), reverse=True))
        _, logps = zip(*sorted(zip(prs_abs[0], logps[0]), reverse=True))
        
        for i, mod in enumerate(mods):
            print('   -> {} ({:.3f}) [{:.3f}]'.format(mod, prs[i], logps[i]))
        
    print('\n')

Number of unique vars across all modalities:  112
112
Category: Alcohol
------------------------------------------------
Var: Alcohol intake frequency. (0.0)
   Modalities:
   -> tbss_FA (0.100) [6.268]
   -> tbss_L3_s (0.093) [5.514]
Var: Alcohol intake frequency. (2.0)
   Modalities:
   -> tbss_FA (0.094) [5.576]
Var: Amount of alcohol drunk on a typical drinking day (0.0)
   Modalities:
   -> tbss_L2 (0.131) [6.991]
   -> tbss_MD (0.120) [5.919]
Var: Frequency of consuming six or more units of alcohol (0.0)
   Modalities:
   -> tbss_MD (0.139) [7.802]
   -> tbss_L2 (0.133) [7.152]
Var: Frequency of drinking alcohol (0.0)
   Modalities:
   -> tbss_MD (0.129) [7.338]
   -> rsfmri_0 (0.129) [7.321]
   -> tbss_MO (0.127) [7.180]
   -> tbss_ISOVF_s (0.127) [7.140]
   -> tbss_L2 (0.127) [7.125]
   -> tbss_FA (0.124) [6.799]
   -> tbss_L3_s (0.121) [6.563]
   -> tbss_FA_s (0.118) [6.250]
   -> rsfmri_10 (0.111) [5.595]


Category: Blood Assays
----------------------------------------------

In [17]:
vars_df_converged = pd.read_pickle('female_vars_testB_convergence9.pkl')

current_mods = vars_df_converged.modality.values

modality_names = [
    'T1_nonlinear_swin_c',           #1
    'T2_nonlinear_swin_c',           #2
    'tbss_FA_swin_c',                #3
    'tbss_MD_swin_c',                #4
    'rsfmri_0_swin_c',               #5
    'swi_swin_c',                    #6
    'tfmri_swin_1_c',                #7
    'tracts_swin_c',                 #8
    'tbss_MO_swin_c',                #9
    'tbss_ISOVF_swin_s_c',           #10
    'tbss_L2_swin_c',                #11
    'rsfmri_2_swin_c',               #12
    'rsfmri_10_swin_c',              #13
    'rsfmri_5_swin_c',               #14
    'rsfmri_21_swin_c',              #15
    'T2_lessions_swin_c',            #16
    'tbss_FA_s_swin_c',              #17
    'tbss_L3_s_swin_c',              #18
]



for idx, mod in enumerate(current_mods):
    vars_df_converged.loc[vars_df_converged["modality"] == mod, "modality"] = modality_names[idx]

vars_over_bThr_list_converged = []

for idx in range(len(vars_df_converged)):
    df = vars_df_converged.iloc[idx].dataframe
    bThr = vars_df_converged.iloc[idx].bonf
    vars_over_bThr = df[df.log_p_values>=bThr].sort_values('log_p_values', ascending=False).names.to_list()
    for var in vars_over_bThr:
        vars_over_bThr_list_converged.append(var)
        
print("Number of unique vars across all modalities: ", len(set(vars_over_bThr_list_converged)))

unique_vars_over_bTHr_converged = list(set(vars_over_bThr_list_converged))
print(len(unique_vars_over_bTHr_converged))

unique_vars_over_bThr_categories_converged = []

for var in unique_vars_over_bTHr_converged:
    idx = list(varsHeader).index(var)
    category = vars_categories[idx]
    unique_vars_over_bThr_categories_converged.append(category)
    
modalities_converged = []
pearson_rs_list_converged = []
logPs_converged = []
for var in unique_vars_over_bTHr_converged:
    modalities_with_var = []
    pearsonRs_with_var = []
    logPs_with_var = []
    for idx in range(len(vars_df_converged)):
        df = vars_df_converged.iloc[idx].dataframe
        modality = vars_df_converged.iloc[idx].modality
        bThr = vars_df_converged.iloc[idx].bonf
        df = df[df.log_p_values>=bThr].sort_values('log_p_values', ascending=False)
        vars_over_bThr = df.names.to_list()
        pearson_r_over_bThr = df.pearson_r.to_list()
        logps_over_bTHr = df.log_p_values.to_list()
        if var in vars_over_bThr:
            modalities_with_var.append(modality)
            pearsonRs_with_var.append(pearson_r_over_bThr[vars_over_bThr.index(var)])
            logPs_with_var.append(logps_over_bTHr[vars_over_bThr.index(var)])
    modalities_converged.append(modalities_with_var)
    pearson_rs_list_converged.append(pearsonRs_with_var)
    logPs_converged.append(logPs_with_var)
    
df_vars_over_bThr_converged = pd.DataFrame.from_dict({
    'names': unique_vars_over_bTHr_converged,
    'Categories': unique_vars_over_bThr_categories_converged,
    'modalities': modalities_converged,
    'pearson_r': pearson_rs_list_converged,
    'log_p_values': logPs_converged
})

for cat in sorted(list(set(df_vars_over_bThr_converged.Categories.to_list()))):
    print("Category: {}".format(cat))
    print('------------------------------------------------')
    names = df_vars_over_bThr_converged[df_vars_over_bThr_converged.Categories == cat].names.to_list()
    for name in sorted(names):
        print('Var: {}'.format(name))
        mods = df_vars_over_bThr_converged[(df_vars_over_bThr_converged.Categories == cat) & (df_vars_over_bThr_converged.names == name)].modalities.to_list()
        prs = df_vars_over_bThr_converged[(df_vars_over_bThr_converged.Categories == cat) & (df_vars_over_bThr_converged.names == name)].pearson_r.to_list()
        logps = df_vars_over_bThr_converged[(df_vars_over_bThr_converged.Categories == cat) & (df_vars_over_bThr_converged.names == name)].log_p_values.to_list()
        print('   Modalities:')
        
        prs_abs = np.abs(prs).tolist()
        _, mods = zip(*sorted(zip(prs_abs[0], mods[0]), reverse=True))
        _, prs = zip(*sorted(zip(prs_abs[0], prs[0]), reverse=True))
        _, logps = zip(*sorted(zip(prs_abs[0], logps[0]), reverse=True))
        
        for i, mod in enumerate(mods):
            print('   -> {} ({:.3f}) [{:.3f}]'.format(mod, prs[i], logps[i]))
        
    print('\n')

Number of unique vars across all modalities:  111
111
Category: Alcohol
------------------------------------------------
Var: Frequency of consuming six or more units of alcohol (0.0)
   Modalities:
   -> tbss_FA_swin_c (0.121) [6.000]
Var: Frequency of drinking alcohol (0.0)
   Modalities:
   -> tbss_MD_swin_c (0.127) [7.130]
   -> tbss_L2_swin_c (0.121) [6.518]
   -> tbss_FA_swin_c (0.117) [6.137]
   -> tbss_FA_s_swin_c (0.112) [5.660]
   -> tbss_ISOVF_swin_s_c (0.110) [5.536]


Category: Blood Assays
------------------------------------------------
Var: C-reactive protein (0.0)
   Modalities:
   -> rsfmri_5_swin_c (-0.096) [5.446]


Category: Cardiac & Circulartory Measurements
------------------------------------------------
Var: Systolic blood pressure, automated reading (0.0)
   Modalities:
   -> T2_lessions_swin_c (0.102) [6.081]
   -> tbss_FA_s_swin_c (0.101) [6.024]
Var: Systolic blood pressure, automated reading (0.1)
   Modalities:
   -> T2_lessions_swin_c (0.100) [5.787]




In [18]:
new_mods = 0

print('BONFERRONI === ', vars_df.iloc[0].bonf)
print('\n')

for cat in sorted(list(set(df_vars_over_bThr.Categories.to_list()))):
    print("Category: {}".format(cat))
    print('------------------------------------------------')
    names = df_vars_over_bThr[df_vars_over_bThr.Categories == cat].names.to_list()
    for name in sorted(names):
        print('Var: {}'.format(name))
        mods = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].modalities.to_list()
        prs = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].pearson_r.to_list()
        logps = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].log_p_values.to_list()
        
        mods0 = df_vars_over_bThr_converged[(df_vars_over_bThr_converged.Categories == cat) & (df_vars_over_bThr_converged.names == name)].modalities.to_list()
        prs0 = df_vars_over_bThr_converged[(df_vars_over_bThr_converged.Categories == cat) & (df_vars_over_bThr_converged.names == name)].pearson_r.to_list()
        logps0 = df_vars_over_bThr_converged[(df_vars_over_bThr_converged.Categories == cat) & (df_vars_over_bThr_converged.names == name)].log_p_values.to_list()
        print('   Modalities:')
        
        prs_abs = np.abs(prs).tolist()
        _, mods = zip(*sorted(zip(prs_abs[0], mods[0]), reverse=True))
        _, prs = zip(*sorted(zip(prs_abs[0], prs[0]), reverse=True))
        _, logps = zip(*sorted(zip(prs_abs[0], logps[0]), reverse=True))
        
        for i, mod in enumerate(mods):
            print('   -> {} ({:.3f}) [{:.3f}]'.format(mod, prs[i], logps[i]))
        
        
        if len(mods0) == 0:
            print('======== ATTENTION! NEW VAR ! ==========')
            new_mods += 1
#         else:
#             prs_abs0 = np.abs(prs0).tolist()
#             _, mods0 = zip(*sorted(zip(prs_abs0[0], mods0[0]), reverse=True))
#             _, prs0 = zip(*sorted(zip(prs_abs0[0], prs0[0]), reverse=True))
        
#             print('   -> {} ({:.3f})'.format(mods0[0], prs0[0]))
        
    print('\n')
    
print('New Mods Missed by SWIN Convergence: ', new_mods)

BONFERRONI ===  5.442009159140952


Category: Alcohol
------------------------------------------------
Var: Alcohol intake frequency. (0.0)
   Modalities:
   -> tbss_FA (0.100) [6.268]
   -> tbss_L3_s (0.093) [5.514]
======== ATTENTION! NEW VAR ! ==========
Var: Alcohol intake frequency. (2.0)
   Modalities:
   -> tbss_FA (0.094) [5.576]
======== ATTENTION! NEW VAR ! ==========
Var: Amount of alcohol drunk on a typical drinking day (0.0)
   Modalities:
   -> tbss_L2 (0.131) [6.991]
   -> tbss_MD (0.120) [5.919]
======== ATTENTION! NEW VAR ! ==========
Var: Frequency of consuming six or more units of alcohol (0.0)
   Modalities:
   -> tbss_MD (0.139) [7.802]
   -> tbss_L2 (0.133) [7.152]
Var: Frequency of drinking alcohol (0.0)
   Modalities:
   -> tbss_MD (0.129) [7.338]
   -> rsfmri_0 (0.129) [7.321]
   -> tbss_MO (0.127) [7.180]
   -> tbss_ISOVF_s (0.127) [7.140]
   -> tbss_L2 (0.127) [7.125]
   -> tbss_FA (0.124) [6.799]
   -> tbss_L3_s (0.121) [6.563]
   -> tbss_FA_s (0.118) [6.250

In [19]:
new_mods = 0

print('BONFERRONI === ', vars_df_converged.iloc[0].bonf)
print('\n')

for cat in sorted(list(set(df_vars_over_bThr_converged.Categories.to_list()))):
    print("Category: {}".format(cat))
    print('------------------------------------------------')
    names = df_vars_over_bThr_converged[df_vars_over_bThr_converged.Categories == cat].names.to_list()
    for name in sorted(names):
        print('Var: {}'.format(name))
        mods = df_vars_over_bThr_converged[(df_vars_over_bThr_converged.Categories == cat) & (df_vars_over_bThr_converged.names == name)].modalities.to_list()
        prs = df_vars_over_bThr_converged[(df_vars_over_bThr_converged.Categories == cat) & (df_vars_over_bThr_converged.names == name)].pearson_r.to_list()
        logps = df_vars_over_bThr_converged[(df_vars_over_bThr_converged.Categories == cat) & (df_vars_over_bThr_converged.names == name)].log_p_values.to_list()
        
        mods0 = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].modalities.to_list()
        prs0 = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].pearson_r.to_list()
        logps0 = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].log_p_values.to_list()
        print('   Modalities:')
        
        prs_abs = np.abs(prs).tolist()
        _, mods = zip(*sorted(zip(prs_abs[0], mods[0]), reverse=True))
        _, prs = zip(*sorted(zip(prs_abs[0], prs[0]), reverse=True))
        _, logps = zip(*sorted(zip(prs_abs[0], logps[0]), reverse=True))
        
        for i, mod in enumerate(mods):
            print('   -> {} ({:.3f}) [{:.3f}]'.format(mod, prs[i], logps[i]))
        
        
        if len(mods0) == 0:
            print('======== ATTENTION! NEW VAR ! ==========')
            new_mods += 1
#         else:
#             prs_abs0 = np.abs(prs0).tolist()
#             _, mods0 = zip(*sorted(zip(prs_abs0[0], mods0[0]), reverse=True))
#             _, prs0 = zip(*sorted(zip(prs_abs0[0], prs0[0]), reverse=True))
        
#             print('   -> {} ({:.3f})'.format(mods0[0], prs0[0]))
        
    print('\n')
    
print('New Mods Missed by CNN Original: ', new_mods)

BONFERRONI ===  5.442009159140952


Category: Alcohol
------------------------------------------------
Var: Frequency of consuming six or more units of alcohol (0.0)
   Modalities:
   -> tbss_FA_swin_c (0.121) [6.000]
Var: Frequency of drinking alcohol (0.0)
   Modalities:
   -> tbss_MD_swin_c (0.127) [7.130]
   -> tbss_L2_swin_c (0.121) [6.518]
   -> tbss_FA_swin_c (0.117) [6.137]
   -> tbss_FA_s_swin_c (0.112) [5.660]
   -> tbss_ISOVF_swin_s_c (0.110) [5.536]


Category: Blood Assays
------------------------------------------------
Var: C-reactive protein (0.0)
   Modalities:
   -> rsfmri_5_swin_c (-0.096) [5.446]


Category: Cardiac & Circulartory Measurements
------------------------------------------------
Var: Systolic blood pressure, automated reading (0.0)
   Modalities:
   -> T2_lessions_swin_c (0.102) [6.081]
   -> tbss_FA_s_swin_c (0.101) [6.024]
Var: Systolic blood pressure, automated reading (0.1)
   Modalities:
   -> T2_lessions_swin_c (0.100) [5.787]


Category: Diet
---

In [20]:
vars_df_converged

,modality,bonf,fdr,dataframe
0,T1_nonlinear_swin_c,5.442009,3.562704,idx ...
1,T2_nonlinear_swin_c,5.442009,3.632591,idx ...
2,tbss_FA_swin_c,5.442009,3.324640,idx ...
3,tbss_MD_swin_c,5.442009,3.537301,idx ...
4,rsfmri_0_swin_c,5.442009,3.681922,idx ...
5,swi_swin_c,5.442009,3.651473,idx ...
6,tfmri_swin_1_c,5.442009,3.533512,idx ...
7,tracts_swin_c,5.442009,3.474175,idx ...
8,tbss_MO_swin_c,5.442009,3.482356,idx ...
9,tbss_ISOVF_swin_s_c,5.442009,3.499444,idx ...


In [21]:
np.power(10, 5.783983)

608111.196940615

In [22]:
vars_df = vars_df.sort_values('modality')

In [23]:
vars_df_converged = vars_df_converged.sort_values('modality')

In [24]:
vars_df_converged

,modality,bonf,fdr,dataframe
0,T1_nonlinear_swin_c,5.442009,3.562704,idx ...
15,T2_lessions_swin_c,5.442009,4.844187,idx ...
1,T2_nonlinear_swin_c,5.442009,3.632591,idx ...
4,rsfmri_0_swin_c,5.442009,3.681922,idx ...
12,rsfmri_10_swin_c,5.442009,3.729379,idx ...
14,rsfmri_21_swin_c,5.442009,3.963276,idx ...
11,rsfmri_2_swin_c,5.442009,3.599822,idx ...
13,rsfmri_5_swin_c,5.442009,3.735161,idx ...
5,swi_swin_c,5.442009,3.651473,idx ...
16,tbss_FA_s_swin_c,5.442009,3.335593,idx ...


In [25]:
vars_df

,modality,bonf,fdr,dataframe
0,T1_nonlinear,5.442009,3.767651,idx ...
5,T2_lesions,5.442009,4.619157,idx ...
4,T2_nonlinear,5.442009,4.016448,idx ...
7,rsfmri_0,5.442009,3.368398,idx ...
17,rsfmri_10,5.442009,3.487095,idx ...
9,rsfmri_2,5.442009,3.333374,idx ...
28,rsfmri_21,5.442009,3.345114,idx ...
12,rsfmri_5,5.442009,3.522064,idx ...
6,swi,5.442009,3.622588,idx ...
48,tbss_FA,5.442009,3.243128,idx ...


In [26]:
for idx in range(len(vars_df)):

    modality = vars_df.iloc[idx].modality
    modalityEN = vars_df_converged.iloc[idx].modality

    
    print(modality, ' VS. ', modalityEN)
    print('----------------------------\n')

    df = vars_df.iloc[idx].dataframe
    bonf = vars_df.iloc[idx].bonf
    df = df[df.log_p_values >= bonf]
    
    
    dfc = vars_df_converged.iloc[idx].dataframe
    bonfC = vars_df_converged.iloc[idx].bonf
    dfc = dfc[dfc.log_p_values >= bonfC]

    print('nIDPs in {}: {}'.format(modality, len(df)))
    print('nIDPs in {}: {}'.format(modalityEN, len(dfc)))
    print('')
    print('Intersection = {}'.format(len(pd.merge(df, dfc, how='inner', on='idx'))))
    print('')
    dfm1 = df[df.idx.isin(list(set(df.idx.to_list()) - set(dfc.idx.to_list())))]
    dfm2 = dfc[dfc.idx.isin(list(set(dfc.idx.to_list()) - set(df.idx.to_list())))]
    print('{}-{} : {}'.format(modality, modalityEN, len(dfm1)))
#     print(dfm1[['idx', 'names', 'Categories', 'pearson_r', 'log_p_values']])
    print('')
    print('{}-{} : {}'.format(modalityEN, modality, len(dfm2)))
#     print(dfm2[['idx', 'names', 'Categories', 'pearson_r', 'log_p_values']])
    
    print('========================================================\n')

T1_nonlinear  VS.  T1_nonlinear_swin_c
----------------------------

nIDPs in T1_nonlinear: 27
nIDPs in T1_nonlinear_swin_c: 40

Intersection = 25

T1_nonlinear-T1_nonlinear_swin_c : 2

T1_nonlinear_swin_c-T1_nonlinear : 15

T2_lesions  VS.  T2_lessions_swin_c
----------------------------

nIDPs in T2_lesions: 6
nIDPs in T2_lessions_swin_c: 8

Intersection = 6

T2_lesions-T2_lessions_swin_c : 0

T2_lessions_swin_c-T2_lesions : 2

T2_nonlinear  VS.  T2_nonlinear_swin_c
----------------------------

nIDPs in T2_nonlinear: 2
nIDPs in T2_nonlinear_swin_c: 32

Intersection = 1

T2_nonlinear-T2_nonlinear_swin_c : 1

T2_nonlinear_swin_c-T2_nonlinear : 31

rsfmri_0  VS.  rsfmri_0_swin_c
----------------------------

nIDPs in rsfmri_0: 56
nIDPs in rsfmri_0_swin_c: 50

Intersection = 50

rsfmri_0-rsfmri_0_swin_c : 6

rsfmri_0_swin_c-rsfmri_0 : 0

rsfmri_10  VS.  rsfmri_10_swin_c
----------------------------

nIDPs in rsfmri_10: 55
nIDPs in rsfmri_10_swin_c: 44

Intersection = 44

rsfmri_10-rsfmr

In [27]:
idx1 = 5
idx2 = 6

modality = vars_df.iloc[idx1].modality
modalityEN = vars_df_converged.iloc[idx2].modality


print(modality, ' VS. ', modalityEN)
print('----------------------------\n')

df = vars_df.iloc[idx1].dataframe
bonf = vars_df.iloc[idx1].bonf
df = df[df.log_p_values >= bonf]


dfc = vars_df_converged.iloc[idx2].dataframe
bonfC = vars_df_converged.iloc[idx2].bonf
dfc = dfc[dfc.log_p_values >= bonfC]

print('nIDPs in {}: {}'.format(modality, len(df)))
print('nIDPs in {}: {}'.format(modalityEN, len(dfc)))
print('')
print('Intersection = {}'.format(len(pd.merge(df, dfc, how='inner', on='idx'))))
print('')
dfm1 = df[df.idx.isin(list(set(df.idx.to_list()) - set(dfc.idx.to_list())))]
dfm2 = dfc[dfc.idx.isin(list(set(dfc.idx.to_list()) - set(df.idx.to_list())))]
print('{}-{} : {}'.format(modality, modalityEN, len(dfm1)))
#     print(dfm1[['idx', 'names', 'Categories', 'pearson_r', 'log_p_values']])
print('')
print('{}-{} : {}'.format(modalityEN, modality, len(dfm2)))
#     print(dfm2[['idx', 'names', 'Categories', 'pearson_r', 'log_p_values']])

print('========================================================\n')

rsfmri_2  VS.  rsfmri_2_swin_c
----------------------------

nIDPs in rsfmri_2: 64
nIDPs in rsfmri_2_swin_c: 55

Intersection = 47

rsfmri_2-rsfmri_2_swin_c : 17

rsfmri_2_swin_c-rsfmri_2 : 8



In [28]:
idx1 = 6
idx2 = 5

modality = vars_df.iloc[idx1].modality
modalityEN = vars_df_converged.iloc[idx2].modality


print(modality, ' VS. ', modalityEN)
print('----------------------------\n')

df = vars_df.iloc[idx1].dataframe
bonf = vars_df.iloc[idx1].bonf
df = df[df.log_p_values >= bonf]


dfc = vars_df_converged.iloc[idx2].dataframe
bonfC = vars_df_converged.iloc[idx2].bonf
dfc = dfc[dfc.log_p_values >= bonfC]

print('nIDPs in {}: {}'.format(modality, len(df)))
print('nIDPs in {}: {}'.format(modalityEN, len(dfc)))
print('')
print('Intersection = {}'.format(len(pd.merge(df, dfc, how='inner', on='idx'))))
print('')
dfm1 = df[df.idx.isin(list(set(df.idx.to_list()) - set(dfc.idx.to_list())))]
dfm2 = dfc[dfc.idx.isin(list(set(dfc.idx.to_list()) - set(df.idx.to_list())))]
print('{}-{} : {}'.format(modality, modalityEN, len(dfm1)))
#     print(dfm1[['idx', 'names', 'Categories', 'pearson_r', 'log_p_values']])
print('')
print('{}-{} : {}'.format(modalityEN, modality, len(dfm2)))
#     print(dfm2[['idx', 'names', 'Categories', 'pearson_r', 'log_p_values']])

print('========================================================\n')

rsfmri_21  VS.  rsfmri_21_swin_c
----------------------------

nIDPs in rsfmri_21: 68
nIDPs in rsfmri_21_swin_c: 45

Intersection = 45

rsfmri_21-rsfmri_21_swin_c : 23

rsfmri_21_swin_c-rsfmri_21 : 0

